## Exporting a MobileNetV2 model to TensorFlow
You can use the Fireball's ```exportToTf``` function to export a model to TensorFlow code. This function creates a
python file that implements the model using TensorFlow APIs. It also creates a numpy file (npz) that contains the parameters of the network. This notebook shows how to use this function to export a Fireball model to TensorFlow. It is assumed that a trained MobileNetV2 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with MobileNetV2](MobileNetV2.ipynb) for more info about using a pretrained MobileNetV2 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)
- [Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)
- [Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='0,1,2,3'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)

# orgFileName = "Models/MobileNetV2.fbm"        # Original model
# orgFileName = "Models/MobileNetV2QR.fbm"      # Quantized - Retrained
# orgFileName = "Models/MobileNetV2PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/MobileNetV2PRQR.fbm"    # Pruned - Retrained - Quantized - Retrained
# orgFileName = "Models/MobileNetV2RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/MobileNetV2RRQR.fbm"    # Reduced - Retrained - Quantized - Retrained
# orgFileName = "Models/MobileNetV2RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained
orgFileName = "Models/MobileNetV2RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/MobileNetV2RRPRQR.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      436        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      279        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      265        
S2_L4_BN         112 112 16 

## Export the model
Fireball creates a folder and puts 2 files in the folder. Here we call the ```exportToTf``` funtion to export the model.

In [4]:
model.exportToTf("Models/MobileNetV2_TF", runQuantized=True, classNames=ImageNetDSet.classNames)



Exporting to TensorFlow model "Models/MobileNetV2_TF" ... 
    Processed all 27 layers.                                     
    Creating parameters file "Params.npz" ... Done.
Done.


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Here we load an image and do the required pre-processing before passing it to the exported model as input. We then print the top-3 most probable predicted labels for the image.

**NOTE**: Please reset the kernel before running the next cell.

In [5]:
import cv2
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order
img = np.float32(img)[..., ::-1]    # Convert to RGB

# Resize the image to 256x256
imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]

# Normalize the image values to the range: -1 .. 1
inputImage = ((np.float32(resizedImg)/127.5)-1.0)

# Choose the module corresponding to the model selected in the first cell of this notebook.
# from Models.MobileNetV2_TF.TfModel import Network
# from Models.MobileNetV2RR_TF.TfModel import Network
from Models.MobileNetV2_TF.TfModel import Network

net=Network()
classNames = net.getClassNames()

classProbs = net.infer([inputImage])[0]
top3Indexes = np.argsort(classProbs)[-3:][::-1]
top3Porbs = classProbs[top3Indexes]
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(classNames[top3Indexes[i]], top3Porbs[i])) 

Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.738154)
    cup (0.190030)
    espresso (0.061312)


## Where do I go from here?

[Exporting MobileNetV2 Model to CoreML](MobileNetV2-CoreML.ipynb)

[Exporting MobileNetV2 Model to ONNX](MobileNetV2-ONNX.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with MobileNetV2](MobileNetV2.ipynb)

[Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)

[Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)

[Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)
